# ch3 網路與節點

前面兩章我們簡單介紹了如何使用nyto建立一個網路，但在這章中，我們將詳細介紹連結節點與建立網路這件事。你將瞭解到有那些技巧可以幫助你更快更好的建立網路。



In [1]:
from nyto import net_tool as to

## 節點技巧

節點是構成網路的一部份，為了使我們能組織出我們需要的網路結構，我們需要學習一些更進階的技巧。

我們會在該小節中介紹:
1. 節點連接界面與節點界面
2. 節點界面間的連接方法
3. 啟動器節點
4. 排序器節點

### 節點連接界面與節點界面

在前面的章節中我們已經使用過節點連接界面來建立網路了，下面我們來看看有哪些方式來取得節點連接界面。

**方法一：**下面是一般取得節點連接界面的方式，也就是在生成網路時會給你一個網路參考跟節點連接界面:

In [2]:
nn, node = to.new_net()
nn   # 網路參考
node # 節點連接界面

<nyto.net.create_connecter.<locals>.node_connecter at 0x7f43e45e36d0>

**方法二：**使用網路本身取得節點連接界面

In [3]:
node = to.create_connecter(nn)
node # 節點連接界面

<nyto.net.create_connecter.<locals>.node_connecter at 0x7f43e45e3340>

而當節點連接界面被指定節點名稱時就會變成節點界面:

In [4]:
node.a # 節點界面

還可以通過節點界面獲得節點名稱與網路參考

In [5]:
node.a.node_id # 節點名稱

'a'

In [6]:
node.a.net_ref # 網路參考

net(mod=set(), data=set())

### 節點界面間的連接方法

除了可以是用數學上的加減乘除之外，也可以導入模型後使用模型的方法或使用函數的呼叫。

**方法一：**數學上的基本運算

In [7]:
nn, node = to.new_net(a=1, b=2)

node.a_add_b = node.a + node.b
node.a_sub_b = node.a - node.b
node.a_mul_b = node.a * node.b
node.a_truediv_b = node.a / node.b

to.get(node.a_add_b, node.a_sub_b, node.a_mul_b, node.a_truediv_b)

(3, -1, 2, 0.5)

**方法二：**導入函數或類

In [8]:
node.func = lambda x: x+1
node.func_return = node.func(node.a)

to.get(node.func_return)

2

In [9]:
class add_something:
    def __init__(self, something):
        self.something=something
    def add(self, x):
        return x+self.something
    
node.obj = add_something(10)
node.obj_return = node.obj.add(node.a)

to.get(node.obj_return)

11

**方法三：**取得相關屬性值

In [10]:
node.data_dict = to.add_data({'a':123, 'b':456, 'c':789})

# 取得方式1
node.get_a_1 = node.data_dict['a']

# 取得方式2
node.key = to.add_data('b')
node.get_a_2 = node.data_dict[node.key]

to.get(node.get_a_1, node.get_a_2)

(123, 456)

In [11]:
class data_class:
    def __init__(self):
        self.a = 123
        self.b = 456
        self.c = 789
        
node.data_obj = to.add_data(data_class())
node.get_obj_a = node.data_obj.a()

to.get(node.get_obj_a)

123

### 啟動器節點

有時候我們建立好一個網路，我們不需要改變它的結構，但是我們需要改變模型的輸入。

例如我們首先要計算:

    c = a + b , a=1, b=2

然後我們要計算:

    c = a + b , a=3, b=4

這時候我們可能會這樣做:

In [12]:
nn, node = to.new_net(a=1, b=2)

node.c = node.a + node.b
print(to.get(node.c))

(node.a, node.b)=(3, 4)
print(to.get(node.c))

3
7


這在普通情況下就是多加幾行代碼的問題，但是如果需要常常切換資料的話，這將會變得非常麻煩。

這在訓練神經網路時很常發生:
* 我們在訓練時，網路需要輸入訓練資料。
* 我們在測試時，網路需要輸入測試資料。

有沒有辦法指定說我標記當我需要訓練時自動使用訓練資料，當我需要測試時自動使用測試資料？

我們這時候就需要使用啟動器:

In [13]:
nn, node = to.new_net(
    case1_a=1, case1_b=2,
    case2_a=3, case2_b=4
)
node.c = node.a + node.b

# 使用啟動器
# case1_c: node.a=node.case1_a, node.b=node.case1_b, 並運行node.c
node.case1_c = nn.launcher(node.c, a=node.case1_a, b=node.case1_b)

# case2_c: node.a=node.case2_a, node.b=node.case2_b, 並運行node.c
node.case2_c = nn.launcher(node.c, a=node.case2_a, b=node.case2_b)

# 運行case1_c定義的動作
print(f"case1_c={to.get(node.case1_c)}")

# 運行case2_c定義的動作
print(f"case2_c={to.get(node.case2_c)}")

case1_c={'c': 3}
case2_c={'c': 7}


可以發現使用啟動器節點的運行結果是一個字典，並將對應結果的節點名稱當作key值。

在上面的例子中我們將不同情況中需要替換掉的值與需要回傳的值，打包成一個啟動節點。當啟動節點被啟動時，啟動節點會替換掉需要替換掉的值，然後回傳需要回傳的值。所有的值都可以是多個，也可以什麼都不做。

需要注意的是，當啟動器被執行時，會真的執行替換的動作。這意味著網路本身會被改變，我們可以從下面的例子看到:

In [14]:
nn, node = to.new_net(a=1)

node.l = nn.launcher(a=10)

print(f"before: a={to.get(node.a)}")

to.get(node.l)

print(f"after: a={to.get(node.a)}")

before: a=1
after: a=10


**啟動器運作邏輯**

起動器的運作基於下面步驟:
1. 取得替換所需的資料
2. 替換指定的節點
3. 執行節點並回傳結果

下面用個例子來說明:

In [15]:
nn, node = to.new_net(a=1, b=2)
node.d = node.c + 4

node.l = nn.launcher(
    node.a, node.b, node.c, node.d, # 執行節點
    a=node.b, b=3, c=node.b         # 替換節點
)

當起動器被啟動時，會執行下面的步驟:
1. 取得node.b的結果: `2`
2. 執行: `node.a=2`, `node.b=3`, `node.c=node.b`
3. 執行: `net_tool.get(ode.a, node.b, node.c, node.d)`

比較需要說明的是替換指定的節點時為什麼*節點a*是替換成2，而*節點c*則是修改連接對象？原因是*節點a*是模型節點，*節點c*是普通節點。而如果替換對象是資料節點則會報錯。

**啟動器時序問題**

啟動器本身是沒有運行先後觀念的，若想要執行嚴格的運行節點的順序是做不到的。

在下面的例子中我們建立了一個*l3啟動器*，我們希望*l3啟動器*能先運行*l2啟動器*再運行*l1啟動器*，這樣我們得到的運行結果應該會是:
    
    {'l1': {'b': 110}, 'l2': {'b': 11}}

In [16]:
nn, node = to.new_net(a=1)

node.b = node.a + 10

node.l1 = nn.launcher(node.b, a=100)
node.l2 = nn.launcher(node.b)

node.l3 = nn.launcher(node.l2, node.l1)

但是我們實際會得到的結果卻是:

In [17]:
to.get(node.l3)

{'l2': {'b': 11}, 'l1': {'b': 110}}

要使得運行順序被嚴格執行，我們需要使用排序器來清楚定義時間的先後順序。

### 排序器節點

使用上面的例子，我們可以改寫成:

In [18]:
nn, node = to.new_net(a=1)

node.b = node.a + 10

node.l1 = nn.launcher(node.b, a=100)
node.l2 = nn.launcher(node.b)

node.o = nn.order(node.l2, node.l1)

運行排序器:

In [19]:
to.get(node.o)

[{'b': 11}, {'b': 110}]

### 批製作啟動器

有時候我們需要使用到批訓練，這需要我們將資料進行打亂排序並切割，並將不同的資料會被分配到不同的啟動器，當不同的啟動器被起動時會導入切割後的資料到網路中。當我們需要手動做到這件事時，會非常麻煩:

In [20]:
import numpy as np

nn, node = to.new_net(
    data=np.array([1,2,3,4,5,6]),
    weight=np.array([-1,2]),
    sum_func=np.sum
)

# 將切割後的資料加總
node.sum = node.sum_func(node.mod_weight*node.batch_data)

# 需要手動切割
node.data_1 = node.data[:2]
node.data_2 = node.data[2:4]
node.data_3 = node.data[4:]

# 製作啟動器
node.sum_1 = nn.launcher(node.sum, batch_data=node.data_1, mod_weight=node.weight)
node.sum_2 = nn.launcher(node.sum, batch_data=node.data_2, mod_weight=node.weight)
node.sum_3 = nn.launcher(node.sum, batch_data=node.data_3, mod_weight=node.weight)

對於上面的狀況可以使用`batch_launcher`批量產生啟動器節點，參數如下:
* nn: 網路來源
* get: 需要取得的輸出節點
* batch_push: 需要切割的導入節點
* static_push: 不需要切割的導入節點，預設為{}
* batch_size: 切割的資料大小
* shuffle: 是否打亂切割，，預設為True

使用上與後面會提到的`push_get`方法類似。

In [21]:
node_list=to.batch_launcher(
    nn=nn,
    get={node.sum, node.batch_data, node.mod_weight},
    batch_push={'batch_data':node.data},
    static_push={'mod_weight':node.weight},
    batch_size=2,
    shuffle=False
)

回傳的結果是裝有啟動器節點的list。

In [22]:
node_list

啟動第一個啟動器節點看看:

In [23]:
to.get(node_list[0])

{'mod_weight': array([-1,  2]), 'sum': 3, 'batch_data': array([1, 2])}

## 網路技巧

下面我們來看看網路的使用技巧，與節點的使用技巧不同，網路的本身主要是用來查看資訊的地方。但也有其他的一些用法，下面將一一說明。

In [24]:
nn, node = to.new_net(
    a = 1,                    # 模型節點
    b = to.add_data(2), # 資料節點
)

node.c = node.a + node.b

### 查看網路的資訊

查看網路的資訊我們使用`info`下的方法成員來查看:

In [25]:
nn.info.unit # 查看單元節點

{'a', 'b'}

In [26]:
nn.info.mod # 查看模型節點

{'a'}

In [27]:
nn.info.data # 查看資料節點

{'b'}

In [28]:
nn.info.node # 查看節點

{'a', 'b', 'c'}

In [29]:
nn.info.is_unit('a') # 判斷節點是否是單元節點

True

In [30]:
nn.info.is_mod('a') # 判斷節點是否是模型節點

True

In [31]:
nn.info.is_data('b') # 判斷節點是否是資料節點

True

In [32]:
nn.info.is_connect_node('c') # 判斷節點是否是連接節點

True

### push_get

有的時候我們想要使用到類似啟動器的功能，但是為了這樣的功能而特別去建立一個啟動器節點又沒有必要時，我們可以使用`net.push_get`方法:

In [33]:
nn, node = to.new_net(a=1)
node.y = node.x1 + node.x2

nn.push_get({'y', 'x1', 'x2'}, x1=node.a, x2=2)

{'x2': 2, 'y': 3, 'x1': 1}

唯一與啟動器不同的地方是輸入的參數由節點界面改成節點名稱的集合。

### 網路清理

當我們長時間使用同一個網路，並在其身上不斷修改時，難免會產生一些已經失去作用的節點，如下圖:

![ch3-1](https://imgur.com/b2J9JqM.png)

我們原先可能是將`x1`和`x3`的相加的值存入`y1`，但後來改成將`x1`和`x2`相乘的值存入`y1`，這樣紀錄相加訊息的節點`0`就失去作用了，但此時該節點仍然存在於網路中，需要手動刪除。或是原本有作用的`y2`和`1`節點後來漸漸用不到了，所以也需要刪除。這時候就可以使用`free_unused_nodes()`函數來進行刪除處理了。

只需要輸入你會用到的幾個節點名稱所組成的集合(set)，neto就會自動整理出與其相關的節點，和與其無關的節點並刪除。下面是使用範例:

In [34]:
# 原來的配置
nn, node = to.new_net(x1=1, x2=2, x3=3)
node.y1 = node.x1 + node.x3
node.y2 = node.x2 * node.x3

# 新加入的配置
node.y1 = node.x1 * node.x2

In [35]:
nn.info.node # 清理前有哪些節點

{'x1', 'x2', 'x3', 'y1', 'y2'}

In [36]:
# 清理掉與 y1節點無關的節點(但不包括模型節點:x3)
to.free_unused_nodes(nn, {'y1'})

{0, 1, 'y2'}

In [37]:
nn.info.node # 清理後有哪些節點

{'x1', 'x2', 'x3', 'y1'}

你可以看到`free_unused_nodes`會回傳刪除的節點名稱。

需要注意的是，為了安全考慮，**單元節點**並不會刪除。如果需要移出網路中的單元節點與其存放的單元，則需要使用`unit.remove`。

In [38]:
nn.unit.remove('x3')

In [39]:
nn.info.node # 清理後有哪些節點

{'x1', 'x2', 'y1'}

## 補充

相信通過前面的介紹，對nyto的了解又更多了。其實nyto不是一個用於優化神經網路模型的工具，對nyto來說任何的運算系統不管是線性還是非線性，可微不可微，神經網路也好，CNN也好，都只不過是一種運算系統。

簡單來說，nyto是用於優化運算系統的工具。只要你能用節點將運算系統組織起來，那麼就可以使用nyto優化。

下面的例子中展示了使用節點搭建了一個計算費氏數列的運算系統，有興趣的話可以實作看看，相信可以對節點的啟動順序，節點被呼叫的時機與次數，會有更深的體悟。

In [40]:
nn, node = to.new_net(x1=1, x2=1)

node.fib = node.n1 + node.n2

node.new_x2 = nn.launcher(node.fib, n1=node.x1, n2=node.x2)['fib']
node.update = nn.launcher(x1=node.x2, x2=node.new_x2)

In [41]:
for i in range(1, 10):
    x1, x2 = to.get(node.x1, node.x2)
    print(f"n={i} fn={x1}")
    
    to.get(node.update)

n=1 fn=1
n=2 fn=1
n=3 fn=2
n=4 fn=3
n=5 fn=5
n=6 fn=8
n=7 fn=13
n=8 fn=21
n=9 fn=34


***
*END*